<a href="https://colab.research.google.com/github/axjasf/YNAB-Categorizer/blob/main/budget.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About

* This project is meant to bring all my personal finance related transactions into one easy to understand view.
* Scope / Value descriptoon
    * ...
* Mechanism
    * It reads CSV files from several US and German banks and Credit Card processors and harmonizes them into one dataframe.
    * It maps fields such as descriptions into payees
        * Lookup mechanism (direct hit and prefix hit) against a payee config JSON file
        * Matching against similarity vectors per payee to identify candidates (manual adjustment of payee JSON afterwards)
    * It categorizes each transaction or splits it into several categories
        * by payee
        * by pre-determination of a percentage split (e.g. for Walgreens that should be sufficient, given that I have categorized transactions since 2014)
        * by semi-automatic order-item review split (e.g. for Apple or Amazon transactions where these files exist and where a split between utility and subscription or grocery, household products or general shopping is of interest)
    * It works with a set of indicator field to mark aspects of interest
        * Indicator for transactions in which automatic determinations have been taken place
        * Task field to address open tasks
        * ...

# Setup

## Loading of Libraries
* Loading of neccessary libraries such as Pandas etc.

In [99]:
import json
import pandas as pd

## Define global Variables
* Create transactions structure that ultimately will hold the transactions dataframes from all bank files
* Create overall transactions dataframe

In [100]:
# Define the transactions dataframe and load the JSON configuration for the different banks
bank_transactions = {}

bank_files = {
        "Chase": "chase.csv",
        "Wells Fargo Checking": "wellsfargo_checking.csv",
        "Apple": "apple.csv",
        "Commerzbank": "commerzbank.csv"
    }

all_transactions = []

# File Conversion

* For each bank file:
    * Load file into individual df
    * Basic quality control on the individual df level
    * Transform columns into target columns
        * Add Bank ID field as well as numberical ID field
    * Add individual df to transactions df

* Special transformations for non-US banks:
    * Date conversion
    * EUR to USD conversion based on an existing file (date and exchange rate or an API call to a free service)

In [101]:
def quality_control(df):
    missing_values = df.isnull().sum()
    column_data_types = df.dtypes

    return missing_values, column_data_types

In [102]:
def adjust_field_names(df, bank=""):

    if 'Category' in df.columns:
        df = df.rename(columns={"Category" : "oldCategory"})

    df.insert(4, 'SplitID',"")
    df.insert(0, 'Date','')
    df.insert(1, 'Payee','')
    df.insert(2, 'Category Type','')
    df.insert(3, 'Category','')
    df.insert(4, 'chkPayee','')
    df.insert(5, 'chkCategory','')
    df.insert(6, 'chkSplit','')
    df.insert(7, 'chkEURUSD','')

#    if bank == "Commerzbank":
#        df.insert("Amount (USD)")
#        df = df.rename(columns={"Booking text" : "Description"})

    return df

## Wells Fargo

### Wells Fargo Checking

In [103]:
# Read the Wells Fargo Checking CSV
bank = 'Wells Fargo Checking'
bank_transactions[bank] = pd.read_csv(bank_files[bank], header=None, names=["Transaction Date", "Amount (USD)", "Status", "Memo", "Description"])

# Adjust field names (if any specific adjustments are required)

# Convert 'Transaction Date' column to datetime
bank_transactions[bank]['Date'] = pd.to_datetime(bank_transactions[bank]['Transaction Date'], errors='coerce')

# Check for problematic dates (rows where the date conversion failed)
problematic_dates = bank_transactions[bank][bank_transactions[bank]['Date'].isna()]

# Perform quality control checks
missing_values, column_data_types = quality_control(bank_transactions[bank])

# Drop columns that are not needed
bank_transactions[bank] = bank_transactions[bank].drop(columns=['Transaction Date', 'Status', 'Memo'])  # Assuming 'Status' is not needed, adjust as necessary

bank_transactions[bank]

,Amount (USD),Description,Date
0,95.05,PAYPAL TRANSFER 230902 1029162881620 AXEL JANSSEN,2023-09-05
1,95.04,PAYPAL TRANSFER 230902 1029162851130 AXEL JANSSEN,2023-09-05
2,29.11,PAYPAL TRANSFER 230902 1029162775449 AXEL JANSSEN,2023-09-05
3,-1200.00,CHECK # 1177,2023-09-01
4,-775.00,RECURRING TRANSFER TO JANSSEN A SAVINGS REF #O...,2023-09-01
...,...,...,...
223,-107.50,VANGUARD BUY INVESTMENT 123022 163602703163938...,2023-01-03
224,-105.00,VANGUARD BUY INVESTMENT 123022 696785703164948...,2023-01-03
225,426.46,MOBILE DEPOSIT : REF NUMBER :915030444929,2023-01-03
226,207.63,MOBILE DEPOSIT : REF NUMBER :815030443890,2023-01-03


## Chase

In [104]:
bank = 'Chase'
bank_transactions[bank] = pd.read_csv(bank_files[bank])

bank_transactions[bank] = adjust_field_names(bank_transactions[bank])

bank_transactions[bank]['Date'] = pd.to_datetime(bank_transactions[bank]['Transaction Date'], errors='coerce')
problematic_dates = bank_transactions[bank][bank_transactions[bank]['Date'].isna()]
missing_values, column_data_types = quality_control(bank_transactions[bank])

bank_transactions[bank] = bank_transactions[bank].drop(columns=['Post Date', 'oldCategory', 'Type', 'Memo', 'Transaction Date'])
bank_transactions[bank] = bank_transactions[bank].rename(columns={"Amount" : "Amount (USD)"})


## Apple

In [105]:
bank = 'Apple'
bank_transactions[bank] = pd.read_csv(bank_files[bank])

bank_transactions[bank] = adjust_field_names(bank_transactions[bank], bank)

bank_transactions[bank]['Date'] = pd.to_datetime(bank_transactions[bank]['Transaction Date'], errors='coerce')
problematic_dates = bank_transactions[bank][bank_transactions[bank]['Date'].isna()]
missing_values, column_data_types = quality_control(bank_transactions[bank])

bank_transactions[bank] = bank_transactions[bank].drop(columns=['Transaction Date', 'Clearing Date', 'Merchant', 'oldCategory', 'Type', 'Purchased By'])

## Commerzbank

In [106]:
bank = 'Commerzbank'
bank_transactions[bank] = pd.read_csv(bank_files[bank])

bank_transactions[bank] = adjust_field_names(bank_transactions[bank])


bank_transactions[bank]['Date'] = pd.to_datetime(bank_transactions[bank]['Transaction date'], errors='coerce', format='%d.%m.%Y') # For Commerzbank, Day.Month.Year
problematic_dates = bank_transactions[bank][bank_transactions[bank]['Date'].isna()]
missing_values, column_data_types = quality_control(bank_transactions[bank])

bank_transactions[bank] = bank_transactions[bank][bank_transactions[bank]['Amount'] != 0]


bank_transactions[bank].insert(7, "Amount (USD)","")

bank_transactions[bank] = bank_transactions[bank].rename(columns={"Booking text" : "Description"})

### EUR to USD conversion
# https://www.wsj.com/market-data/quotes/fx/EURUSD/historical-prices

exchange_rates_data = pd.read_csv('eur_usd_exchange_rates.csv')

# Convert the date columns to consistent datetime format
exchange_rates_data['Date'] = pd.to_datetime(exchange_rates_data['Date'], format='%m/%d/%Y')

# Merge on the date columns to add the exchange rate to bank_transactions[bank]
bank_transactions[bank] = bank_transactions[bank].merge(exchange_rates_data[['Date', ' Close']], on='Date', how='left')

# Add the chkEURUSD column based on the ' Close' column value
bank_transactions[bank]['chkEURUSD'] = np.where(bank_transactions[bank][' Close'].isna(), 'E', 'A')

# Convert the Amount from EUR to USD
bank_transactions[bank]['Amount (USD)'] = bank_transactions[bank]['Amount'] * bank_transactions[bank][' Close']

# Drop the ' Close' column as it's not needed anymore in bank_transactions[bank]
bank_transactions[bank].drop(' Close', axis=1, inplace=True)

### End of currency conversion

#bank_transactions[bank].drop(bank_transactions[bank].columns[[16, 15, 14, 13, 12, 10, 9, 8]], axis=1, inplace=True)
bank_transactions[bank] = bank_transactions[bank].drop(columns=['Transaction date', 'Value date', 'Transaction type', 'Amount', 'Account of initiator', 'Bank code of account of initiator', 'IBAN of account of initiator'])


In [107]:
all_transactions = pd.concat(bank_transactions, keys=bank_transactions.keys())
all_transactions['Account-ID'] = all_transactions.index.get_level_values(0) + "-" + all_transactions.index.get_level_values(1).astype(str)

# Payees

## Payee Harmonization

In [108]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

class MerchantMatcher:
    def __init__(self, data_df):
        self.data = data_df
        self.vectorizer = self._train_vectorizer()
        self.payee_vectors = self._compute_payee_vectors()
        self.positive_list_descriptions = self._get_positive_list_descriptions()

    def _match_prefix(self, description, merchant_details):
        prefix_length = merchant_details.get('Prefix Length', 50)
        for known_description in merchant_details['Positive List']:
            truncated_payee = known_description.lower()[:prefix_length]
            if description.lower().startswith(truncated_payee):
                return True
        return False


    def _train_vectorizer(self):
        all_descriptions = [desc.lower() for descriptions in self.data['Positive List'] for desc in descriptions]
        return TfidfVectorizer().fit(all_descriptions)

    def _compute_payee_vectors(self):
        payee_vectors = {}
        for merchant, details in self.data.iterrows():
            tfidf_matrix = self.vectorizer.transform([desc.lower() for desc in details['Positive List']])
            avg_vector = np.asarray(tfidf_matrix.mean(axis=0))
            payee_vectors[merchant] = avg_vector
        return payee_vectors

    def _get_positive_list_descriptions(self):
        return set(desc.lower() for descriptions in self.data['Positive List'] for desc in descriptions)

    def predict_payees(self, transaction_df):
        mg_values = []
        chkpayee_values = []
        candidates = []

        for _, row in transaction_df.iterrows():
            description_lower = row['Description'].lower() if row['Description'] else None
            current_merchant = None
            current_chkpayee = None

            if pd.isna(description_lower) or not description_lower.strip():
                mg_values.append(None)
                chkpayee_values.append(None)
                continue

            for merchant, details in self.data.iterrows():
                if description_lower in [desc.lower() for desc in details['Positive List']]:
                    current_merchant = merchant
                    current_chkpayee = 'A'
                    break

                # Check for prefix matching
                if self._match_prefix(description_lower, details):
                    current_merchant = merchant
                    current_chkpayee = 'P'
                    break

            if not current_merchant:
                description_vector = self.vectorizer.transform([description_lower])
                similarities = {merchant: linear_kernel(description_vector, np.asarray(vector))[0][0] for merchant, vector in self.payee_vectors.items()}
                predicted_merchant = max(similarities, key=similarities.get)
                max_similarity = similarities[predicted_merchant]

                if max_similarity > self.data.loc[predicted_merchant, 'Threshold']:
                    candidates.append({'Payee': predicted_merchant, 'Description': row['Description'], 'Probability': max_similarity})

            mg_values.append(current_merchant)
            chkpayee_values.append(current_chkpayee or 'C')

        transaction_df['Payee'] = mg_values
        transaction_df['chkPayee'] = chkpayee_values
        candidates_df = pd.DataFrame(candidates)
        return transaction_df, candidates_df




# Sample Usage
data_df = pd.read_json("payee_matching.json", orient="index")  # Replace with your DataFrame loading mechanism

matcher = MerchantMatcher(data_df)
payees_identified_df, payees_candidates_df = matcher.predict_payees(all_transactions)
payees_identified_df = payees_identified_df[payees_identified_df['chkPayee'] != 'C']

file_payees_identified = "z_payees_identified.csv"
file_payees_candidates = "z_payees_candidates.csv"

if os.path.exists(file_payees_identified): os.remove(file_payees_identified)
if os.path.exists(file_payees_candidates): os.remove(file_payees_candidates)
payees_identified_df.to_csv(file_payees_identified, index=False)
payees_candidates_df.to_csv(file_payees_candidates, index=False)



# Categories

* Transactions <--> Payee mapping (1:1)
* Transactions <--> Amazon Orders mapping and splitting
* Transactions <--> Apple Orders mapping and splitting
* Transactions <--> Walgreens splitting

## Direct assignment

In [109]:
# Transactions <--> Payee mapping (1:1)

with open('payee_matching.json', 'r') as file:
    payee_data = json.load(file)

# List to hold split transactions
split_transactions = []

# Iterate over each row in the dataframe
for idx, row in all_transactions.iterrows():
    payee = row['Payee']

    # Check if payee exists in the JSON data
    if payee in payee_data:
        categories = payee_data[payee]['Categories']

        # If no category exists, update the row's category columns
        if len(categories) == 0:
            all_transactions.at[idx, 'chkCategory'] = 'E'

        # If only one category exists, update the row's category columns
        if len(categories) == 1:
            all_transactions.at[idx, 'Category Type'] = categories[0]['Category Type']
            all_transactions.at[idx, 'Category'] = categories[0]['Category']
            all_transactions.at[idx, 'chkCategory'] = 'A'

        # If multiple categories exist, create split transactions
        elif len(categories) > 1:
            all_transactions.at[idx, 'Category Type'] = ''  # Empty the master row's category columns
            all_transactions.at[idx, 'Category'] = ''
            all_transactions.at[idx, 'SplitID'] = str(row['Account-ID']) + '-' + 'M'
            all_transactions.at[idx, 'chkCategory'] = 'A'

            for idx_split, category in enumerate(categories, start=1):
                new_row = row.copy()
                new_row['Category Type'] = category['Category Type']
                new_row['Category'] = category['Category']
                new_row['SplitID'] = str(row['Account-ID']) + '-' + 'S' + str(idx_split-1)
                new_row['chkCategory'] = 'A'

                # Update the 'Amount (USD)' based on the percentage split from the JSON
                new_row['Amount (USD)'] = row['Amount (USD)'] * category.get('Percentage', 1)

                split_transactions.append(new_row)

# Append the split transactions to the main dataframe
all_transactions = pd.concat([all_transactions, pd.DataFrame(split_transactions)], ignore_index=False)

## Amazon categorization

1. **Identification of Amazon Transactions**:
* Filter transactions with the Payee set to "Amazon" or "Amazon Grocery".
* From this subset, take those transactions that don't already have a chkCategory flag.
   
2. **Match the Transactions to Orders**:
   - For each identified Amazon transaction, we need to match it with an order from the Amazon order file. This matching will be based on the transaction date (with a tolerance of a few days) and the payment amount.
   
3. **Extract Items for the Matched Orders**:
   - Once we have identified the matching order, we will then look up the items related to that order from the Amazon order items file.
   
4. **Categorize the Items**:
   - We will categorize the items into two groups:
     - Groceries (Split 1)
     - All other line items (Split 2...n)
   
5. **Modify the Transactions**:
   - We will then modify the transactions to reflect these splits, updating the description for each split with the appropriate line item description.

In [ ]:
import re
import pandas as pd

class AmazonProcessor:
    def __init__(self, orders_headers, order_items=None, grocery_keywords=None):
        self.orders_headers = orders_headers
        self.order_items = order_items
        self.grocery_keywords = grocery_keywords
        self.processed_orders = self.split_multiple_payments(orders_headers)

    @staticmethod
    def extract_multiple_payments(payment_string):
        # Match both types: "2023-08-04: $14.98" and "MasterCard ending in 3490: August 23, 2023: $39.08"
        pattern = r"(?:MasterCard|Visa)?(?:\s*ending\s*in\s*\d{4})?:?\s*([A-Za-z]*\s*\d{1,2},?\s*\d{4}|\d{4}-\d{1,2}-\d{1,2})[^$]*\$\s*([\d,]+\.\d{2})"
        matches = re.findall(pattern, payment_string)

        # Convert month names to month numbers for consistent date format
        month_names = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
        month_map = {month: str(index + 1) for index, month in enumerate(month_names)}

        # Process the dates to have a consistent format
        processed_matches = []
        for date, amount in matches:
            for month, month_num in month_map.items():
                date = date.replace(month, month_num)
            date = date.replace(",", "").replace(" ", "-")
            processed_matches.append((date, float(amount.replace(',', ''))))

        return processed_matches

    def split_multiple_payments(self, df):
        new_rows = []
        for _, row in df.iterrows():
            payments = self.extract_multiple_payments(row['payments'])
            for date, amount in payments:
                new_row = row.copy()
                new_row['payment_date'] = date
                new_row['payment_amount'] = amount
                new_rows.append(new_row)
        return pd.DataFrame(new_rows)

    def get_processed_orders(self):
        return self.processed_orders.copy()

# Sample Usage in Google Colab:
# Assuming you've uploaded 'amazon_axel_orders_headers_jul_aug.csv'
orders_headers_df = pd.read_csv("/content/amazon_axel_orders_headers_jul_aug.csv")
amazon_processor = AmazonProcessor(orders_headers_df)
order_data_processed = amazon_processor.get_processed_orders()

order_data_processed



In [153]:
class AmazonTransactionMatcher:
    def __init__(self, order_data, transaction_data):
        self.order_data = order_data
        self.transaction_data = transaction_data
        self.matches = pd.DataFrame()

    def match_transactions(self):
        # Convert 'payment_date' in order_data to datetime format
        self.order_data['payment_date'] = pd.to_datetime(self.order_data['payment_date'])

        # Merge based on payment_date and payment_amount
        self.matches = pd.merge(
            left=self.transaction_data,
            right=self.order_data,
            left_on=['Date', 'Amount (USD)'],
            right_on=['payment_date', 'payment_amount'],
            how='inner'
        )

    def get_matched_transactions(self):
        return self.matches.copy()


# Assuming 'all_transactions' is the DataFrame containing your transactions
matcher = AmazonTransactionMatcher(processed_orders, all_transactions)
matcher.match_transactions()
matched_transactions = matcher.get_matched_transactions()
print(matched_transactions.head())


Empty DataFrame
Columns: [Date, Account-ID, SplitID, Payee, Category Type, Category, Amount (USD), Description, chkPayee, chkCategory, chkEURUSD, order id, items, to, date, total, shipping, shipping_refund, gift, tax, refund, payments, payment_date, payment_amount]
Index: []

[0 rows x 24 columns]


# Output

## Dataframe preparation

In [110]:
# Reorder Columns

all_transactions = all_transactions[[
    'Date',
    'Account-ID',
    'SplitID',
    'Payee',
    'Category Type',
    'Category',
    'Amount (USD)',
    'Description',
    'chkPayee',
    'chkCategory',
    'chkEURUSD']]

# Sort rows
all_transactions = all_transactions.sort_values(by=['Date', 'Account-ID', 'SplitID'], ascending=[False, True, True])

# Formating
all_transactions['Amount (USD)'] = all_transactions['Amount (USD)'].round(2)
all_transactions['Amount (USD)'] = all_transactions['Amount (USD)'].apply(lambda x: "${:,.2f}".format(x))


## Output file generation

In [111]:
if os.path.exists("z_output.csv"): os.remove("z_output.csv")
all_transactions.to_csv("z_output.csv", index=False)